In [13]:
import subprocess
import os
from datetime import datetime

distances = [12, 14]
cwd = os.getcwd()
base_dir = os.path.join(cwd, "data/treshold_plots")
os.makedirs(base_dir, exist_ok=True)

existing_ids = [
    int(f.split("-")[0]) for f in os.listdir(base_dir) 
    if os.path.isdir(os.path.join(base_dir, f)) and f.split("-")[0].isdigit()
]

next_id = max(existing_ids, default=0) + 1
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = os.path.join(base_dir, f"{next_id}-{timestamp}")
os.makedirs(output_dir, exist_ok=True)

for d in distances:
    output_file = os.path.join(output_dir, f"d_{d}.txt")
    command = f"/home/tommaso-peduzzi/Documents/clayg/cmake-build-debug/clayg {d} {d} 0 0.06 {output_file} p_step=0.001 mode=compare dump=false"
    subprocess.Popen(["xterm", "-e", f'cd "{cwd}" && {command}; bash'])


In [21]:
import glob
import re
import os
import sys
import plotly.graph_objects as go

if len(sys.argv) < 2:
    print("Usage: python script.py <id1> <id2> ...")
    sys.exit(1)

plot_ids = [1,2]
smoothing_factor = 0.5
union_colors = [
    "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#4292c6",
    "#3182bd", "#1f77b4", "#2171b5", "#08519c", "#08306b"]

clayg_colors = [
    "#fdae6b", "#ffbb78", "#ff8c00", "#fd8d3c", "#ffa726",
    "#f16913", "#ff7f0e", "#d95f02", "#d94801", "#a63603"]

fig = go.Figure()

title = f"Threshold Plot for runs"

for plot_id in plot_ids:
    data_folder = os.path.join("data/treshold_plots", f"{plot_id}-*")
    matching_folders = glob.glob(data_folder)
    if not matching_folders:
        print(f"No folder found with ID {plot_id}")
        continue

    folder_name = os.path.basename(matching_folders[0])
    title += ", " + folder_name
    data_by_depth = {}
    files = glob.glob(os.path.join(matching_folders[0], "d=*.txt"))
    pattern = re.compile(r"d=(\d+)\.txt")

    for filename in files:
        match = pattern.search(os.path.basename(filename))
        if not match:
            continue
        depth = int(match.group(1))
        p_list, uf_list, clayg_list = [], [], []
        with open(filename, "r") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) < 3:
                    continue
                try:
                    p_list.append(float(parts[0]))
                    uf_list.append(float(parts[1]))
                    clayg_list.append(float(parts[2]))
                except ValueError:
                    continue
        data_by_depth[depth] = {"p": p_list, "union_find": uf_list, "clayg": clayg_list}

    depths = sorted(data_by_depth.keys())

    for i, depth in enumerate(depths):
        ddata = data_by_depth[depth]
        union_color = union_colors[i % len(union_colors)]
        clayg_color = clayg_colors[i % len(clayg_colors)]
        
        fig.add_trace(go.Scatter(
            x=ddata["p"], y=ddata["union_find"], mode="lines+markers",
            name=f"Union Find d={depth} ({folder_name})",
            line=dict(color=union_color, shape="spline", smoothing=smoothing_factor),
            legendgroup=f"Union Find {plot_id}",
            legendgrouptitle_text=f"Union Find ({folder_name})",
        ))

        fig.add_trace(go.Scatter(
            x=ddata["p"], y=ddata["clayg"], mode="lines+markers",
            name=f"ClAYG d={depth} ({folder_name})",
            line=dict(color=clayg_color, shape="spline", smoothing=smoothing_factor),
            legendgroup=f"ClAYG {plot_id}",
            legendgrouptitle_text=f"ClAYG ({folder_name})",
        ))

fig.update_layout(
    title=title,
    xaxis_title="Physical Error Rate (p)",
    yaxis_title="Logical Error Rate (l)",
    template="plotly_white",
    hoverlabel_namelength=-1,
    legend=dict(groupclick="toggleitem")
)

fig.show(renderer="browser")
